### Connect to Cognite with OIDC

CDF UI:
- https://hafslundeco.fusion.cognite.com/
- heco-dev

Cognite Python SDK:
- https://cognite-docs.readthedocs-hosted.com/projects/cognite-sdk-python/en/latest/
- `pip install cognite-sdk msal`


In [2]:
import os
import pandas as pd
import numpy as np

from datetime import datetime
from msal import PublicClientApplication

from cognite.client import CogniteClient
from cognite.client.data_classes import TimeSeries, Asset

In [3]:
# Log-in detaljer
TENANT_ID = str(os.getenv("AZURE_TENANT_ID")) # Disse må endres til string ved å sette
CLIENT_ID = str(os.getenv("AZURE_CLIENT_ID")) # str() rundt, ellers kræsjer neste blokk
CDF_CLUSTER = "az-power-no-northeurope"
COGNITE_PROJECT = "heco-dev"

In [4]:
# Code to log-in WIHTOUT client_secret
SCOPES = [f"https://{CDF_CLUSTER}.cognitedata.com/.default"]

AUTHORITY_HOST_URI = "https://login.microsoftonline.com"
AUTHORITY_URI = AUTHORITY_HOST_URI + "/" + TENANT_ID
PORT = 53000


def authenticate_azure():

    app = PublicClientApplication(client_id=CLIENT_ID, authority=AUTHORITY_URI)

    # interactive login - make sure you have http://localhost:port in Redirect URI in App Registration as type "Mobile and desktop applications"
    creds = app.acquire_token_interactive(scopes=SCOPES, port=PORT)
    return creds


creds = authenticate_azure()

client = CogniteClient(
    token_url=creds["id_token_claims"]["iss"],
    token=creds["access_token"],
    token_client_id=creds["id_token_claims"]["aud"],
    project=COGNITE_PROJECT,
    base_url=f"https://{CDF_CLUSTER}.cognitedata.com",
    client_name="cognite-python-dev",
)

print(client.iam.token.inspect())

{
    "subject": "-sTDaBmpTYrnewchD-RUzELOvmjGCp2kx7imzOOWmTg",
    "projects": [
        {
            "url_name": "heco-dev",
            "groups": [
                2296494088654948,
                3141301975681982,
                5972473111747815
            ]
        }
    ],
    "capabilities": [
        {
            "assetsAcl": {
                "version": 1,
                "actions": [
                    "READ"
                ],
                "scope": {
                    "datasetScope": {
                        "ids": [
                            "782671167763856",
                            "788450409833218",
                            "1020945958129838",
                            "1349361264490416",
                            "1612776730017397",
                            "1743179964318084",
                            "2237225820987525",
                            "3065916339847684",
                            "4509487486068549",
                        

In [5]:
client.time_series.list(limit=1)

,id,external_id,name,is_string,metadata,unit,is_step,security_categories,data_set_id,created_time,last_updated_time
0,14234420201,SHOP_market.1.price_1651152286618,1 market price,False,"{'shop:watercourse': 'OE', 'shop:attribute': '...",EUR/MWh,True,[],3065916339847684,1651152312622,1651152312622


### Generelle tips
- Sjekk at du er i riktig prosjekt: `heco-dev`
- Sjekk at du bruker riktig datasett: `uc:006:ml_test`
- Sjekk datamodell / assethierarki i GUI
    - Den store fordelen med CDF er at du koble sammen forskjellige datakilder og kontekstualisere dem, dvs. sette dem i sammenheng
- Husk å konvertere alle tidsstempler til UTC

### Playaround
- Create a new asset
- Create a new timeseries
- Write to timeseries
- Read data from timeseries

In [34]:
# Create an asset
asset_name = "NO1_test2"
client.assets.create([Asset(
    external_id=asset_name,
    name=asset_name,
    data_set_id=client.data_sets.retrieve(external_id="uc:006:ml_test").id,
)])

,external_id,name,data_set_id,metadata,id,created_time,last_updated_time,root_id
0,NO1_test2,NO1_test2,788450409833218,{},8330732854232942,1656504839448,1656504839448,8330732854232942


In [38]:
# Delete an asset
asset_name = "NO1_test2"
client.assets.delete(
    external_id=asset_name,
)

In [35]:
# Create a new timeseries connected to asset `NO1`
forecast_date = datetime(2022, 5, 20, 12, 0, 0, 0).replace(microsecond=0)
name = f"NO1_generation_test_series_2{forecast_date}"

client.time_series.create(
    time_series=TimeSeries(
        external_id=name,
        asset_id=client.assets.retrieve(external_id="NO1_test2").id,
        name=name,
        unit="MWh/h",
        data_set_id=client.data_sets.retrieve(external_id="uc:006:ml_test").id
    )
)

,value
id,4876105227349445
external_id,NO1_generation_test_series_22022-05-20 12:00:00
name,NO1_generation_test_series_22022-05-20 12:00:00
is_string,False
unit,MWh/h
asset_id,8330732854232942
is_step,False
security_categories,[]
data_set_id,788450409833218
created_time,1656504843224


In [6]:
# Write datapoints to timeseries
# Tuple of datetime and value, remember time is given in UTC

import pandas as pd
from datetime import datetime

df_wind = pd.read_csv("vinddata_2022.csv")

datapoints = []

df_wind.set_index("Tid(norsk normaltid)",inplace=True)

In [9]:
df_wind.tail(50)

,Høyeste middelvind (1 t),Middelvind
Tid(norsk normaltid),,
2022-04-26 23:00:00,4.2,3.9
2022-04-27 00:00:00,5.0,4.6
2022-04-27 01:00:00,5.1,4.8
2022-04-27 02:00:00,4.8,4.6
2022-04-27 03:00:00,4.8,3.0
2022-04-27 04:00:00,3.4,3.2
2022-04-27 05:00:00,4.1,4.0
2022-04-27 06:00:00,4.3,2.8
2022-04-27 07:00:00,5.2,5.2


In [59]:
for i in range(len(df_wind.index)):
    if str(df_wind.index[i])[:-6] == "2022-01-01 00:00:00" or str(df_wind.index[i])[:-6] == "2013-10-25 17:00:00":
        continue
    index = datetime.strptime(str(df_wind.index[i]), '%Y-%m-%d %H:%M:%S')
    # if df_wind.loc[str(index) + "+01:00",'wind_forecast'] != "-":
    datapoint = float(df_wind.loc[str(index),'Middelvind'])
    datatuple = (index,datapoint)
    datapoints.append(datatuple)

# print(datapoints)
# datapoints = [
#     (datetime(2022,6,20,9), 0),
#     (datetime(2022,6,20,9,58), 5),
#     (datetime(2022,6,20,10), 30),
#     (datetime(2022,6,20,10,50), 24),
#     (datetime(2022,6,20,11,50), 0),
# ]   

res = client.time_series.retrieve(external_id="NO1_wind_per_15min")
res.unit = "m/s"
# res = client.time_series.update(res)

print(client.time_series.retrieve(external_id="NO1_wind_per_15_min_test").id)

client.datapoints.insert(datapoints, id=client.time_series.retrieve(external_id="NO1_wind_per_15_min_test").id)

866823001927274


TypeError: update() got an unexpected keyword argument 'id'

In [37]:
# Get datapoints in pandas dataframe
df = client.datapoints.retrieve_dataframe(
    start=datetime(2022, 6, 20),
    end=datetime(2022, 6, 21),
    aggregates=["average", "sum"],
    granularity="1h",
    id=client.time_series.retrieve(external_id="NO1_generation_test_series_2022-05-20 12:00:00").id,
)
print(df)

                     2661217891469451|average  2661217891469451|sum
2022-06-20 09:00:00                       3.0                   5.0
2022-06-20 10:00:00                      27.0                  54.0
2022-06-20 11:00:00                       0.0                   0.0
